# Preparing The Environment
Importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
import re

import warnings
warnings.filterwarnings("ignore")

# Loading The Data

In [2]:
# Reading The target Excel file
df = r'My port - sheet location - Copy\MyPoert AUG sheet1.xlsx'

# Save all sheets in the excel book in a dictionary
data_frames = {}
with pd.ExcelFile(df) as xl:
  for sheet_name in xl.sheet_names:
    data_frames[sheet_name] = pd.read_excel(xl, sheet_name)

# Inspecting the data
to get fimiliar with its structure and content

In [3]:
print(data_frames.keys())

dict_keys(['Reunion', 'Paramaribo', 'Isle of Man', 'San Marino', 'Majuro', 'Guyana', 'Saint Kitts and Nevis', 'Guernsey', 'Cayman Islands', 'Dominica', 'Fiji', 'Greenland', 'Turks and Caicos Islands'])


In [ ]:
for country in data_frames.keys():
  print(data_frames[country].info())

In [5]:
data_frames['Fiji'].head()

,trip,price offer,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,From Cádiz to Miami Beach,Trip:ONE WAY\nPrice starts from EUR: 613,NaN,NaN,NaN,"""CLAUSES AND TERMS""\n\nOffer is available for ..."
1,NaN,NaN,NaN,NaN,NaN,NaN
2,From Berwyn to Miami Beach,Trip: ROUND TRIP\nPrice starts from EUR:613,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,from Bilbao to Miami Beach,Trip:ONE WAY\nPrice starts from EUR :613,NaN,NaN,NaN,NaN


# Standardizing Dataframes Structure

In [6]:
for country in data_frames.keys():

# Selecting the first 2 columns
  data_frames[country] = data_frames[country].iloc[:, :2]

# Standardizing columns names
  data_frames[country].columns = ['trip', 'price_offer']

# Handling merged cells
  data_frames[country]['price_offer2'] = \
      data_frames[country]['price_offer'] \
    + data_frames[country]['price_offer'].shift(-1).replace({None: ''})

# Dropping unnecessary rows
  data_frames[country] = data_frames[country][data_frames[country]['trip'].notna()]

# Reseting index after dropping rows
  data_frames[country].reset_index(drop=True, inplace= True)

# Distinguishing countries through a country column
  data_frames[country]['country'] = country

# Concatenating Dataframes
without reseting the index to detect errors more efficiently

In [7]:
semi_raw_df = pd.concat([data_frames[dataframe] for dataframe in data_frames.keys()])
semi_raw_df.head()

,trip,price_offer,price_offer2,country
0,FROM Frederikshavn TO Liepaja,Trip: ONE WAY\nPrice starts from:EUR 3306\n,Trip: ONE WAY\nPrice starts from:EUR 3306\nTr...,Reunion
1,from Emden TO Fos-sur-Mer,Trip:ONE WAY\nprice start from :EUR 6363,Trip:ONE WAY\nprice start from :EUR 6363,Reunion
2,FROM Zeebrugge TO Bensersiel,Trip:ONE WAY\nprice start from :EUR 3666,Trip:ONE WAY\nprice start from :EUR 3666,Reunion
3,FROM Emdan TO Sagunto,Trip:ONE WAY\nprice start from :EUR 6166,Trip:ONE WAY\nprice start from :EUR 6166,Reunion
4,FROM Emdan TO Milazzo,Trip:ONE WAY\nprice start from :EUR 3161,Trip:ONE WAY\nprice start from :EUR 3161,Reunion


In [8]:
semi_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 180 entries, 0 to 17
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   trip          180 non-null    object
 1   price_offer   180 non-null    object
 2   price_offer2  180 non-null    object
 3   country       180 non-null    object
dtypes: object(4)
memory usage: 7.0+ KB


# Preparing The Dataframe
Extracting features in a suitable form by standardizing content

In [9]:
for col in semi_raw_df.columns:
  semi_raw_df[col] = semi_raw_df[col].str.lower()

# Splitting trips
semi_raw_df['port_from'] = semi_raw_df['trip']\
  .apply(lambda x: x.split("to ")[0].replace("from ", "").strip())

semi_raw_df['port_to'] = semi_raw_df['trip']\
  .apply(lambda x: x.split("to ")[1].strip())

# Extracting trip type 
def extract_trip_type(text):
  if text.find('one') != -1:
    return 'O'
  elif text.find('round') != -1:
    return 'R'
  else:
    return 'Unknown'
semi_raw_df['trip_type'] = semi_raw_df['price_offer']\
  .apply(extract_trip_type)

# Extracting price
pattern = r"\d+"
# This function extracts the first digit sequence
def extract_first_digits(text):
  match = re.search(pattern, text)
  return match.group(0) if match else None

semi_raw_df['price'] = semi_raw_df['price_offer2']\
  .apply(extract_first_digits).astype('int64')


aug_sheet = semi_raw_df[['country', 'port_from', 'port_to', 'trip_type', 'price']]

aug_sheet.head()

,country,port_from,port_to,trip_type,price
0,reunion,frederikshavn,liepaja,O,3306
1,reunion,emden,fos-sur-mer,O,6363
2,reunion,zeebrugge,bensersiel,O,3666
3,reunion,emdan,sagunto,O,6166
4,reunion,emdan,milazzo,O,3161


In [10]:
aug_sheet.info()

<class 'pandas.core.frame.DataFrame'>
Index: 180 entries, 0 to 17
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   country    180 non-null    object
 1   port_from  180 non-null    object
 2   port_to    180 non-null    object
 3   trip_type  180 non-null    object
 4   price      180 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 8.4+ KB


# Loading & preparing The Backend Files

In [11]:
countries_ids = pd.read_excel(r'My port - Copy\Countries IDs.xlsx')

# Standardize content
countries_ids.columns = ['country', 'country_id']
countries_ids['country'] = countries_ids['country'].apply(lambda x: x.lower())

countries_ids.head()

,country,country_id
0,reunion,100
1,suriname,40
2,montenegro,38
3,cape verde,44
4,western sahara,62


In [12]:
countries_ids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   country     71 non-null     object
 1   country_id  71 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.2+ KB


In [13]:
port_cities = pd.read_excel(r'My port - Copy\Ports_ceties_codes_and_IDs.xlsx')

# Standardize content
port_cities.columns = ['city', 'port_name', 'port_code', 'port_id']
for col in ['city', 'port_name', 'port_code']:
  port_cities[col] = port_cities[col].apply(lambda x: x.lower())

port_cities.head()

,city,port_name,port_code,port_id
0,guttenberg,a coruña,ogx,870
1,union city,aalborg,ffo,229
2,west new york,algeciras,beh,765
3,hoboken,alicante,zkn,843
4,guttenberg,qweeeee,ogx,870


In [14]:
port_cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   city       105 non-null    object
 1   port_name  105 non-null    object
 2   port_code  105 non-null    object
 3   port_id    105 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 3.4+ KB


# Checking for Possible Errors 

## Checking for duplicates


In [15]:
# Creating a dictionary for possible duplicates
duplicates = {'file_name': [], 'column_name': [], 'duplicates_rows': []}

### countries_ids

In [16]:
for col in countries_ids.columns:
  errors = countries_ids[countries_ids[col].duplicated(keep= False)]
  print(col + " duplicates: " + str(len(errors)))
  if len(errors) > 0:
    duplicates["file_name"].append('Countries IDs')
    duplicates["column_name"].append(col)
    duplicates["duplicates_rows"].append((errors.index.values) + 2)

country duplicates: 2
country_id duplicates: 2


### port_cities

In [17]:
for col in port_cities.columns:
  if col != 'city':
    errors = port_cities[port_cities[col].duplicated(keep= False)]
    print(col + " duplicates: " + str(len(errors)))
    if len(errors) > 0:
      duplicates["file_name"].append('Ports_ceties_codes_and_IDs')
      duplicates["column_name"].append(col)
      duplicates["duplicates_rows"].append((errors.index.values) + 2)

port_name duplicates: 4
port_code duplicates: 6
port_id duplicates: 6


In [18]:
duplicates_df = pd.DataFrame(duplicates, index=np.arange(len(duplicates['file_name'])))

city duplicates were skipped as they aren't considered as errors.

## Checking for typos

In [19]:
# Creating a dictionary for possible typos
typos = {'file_name': [], 'column_name': [], 'typos_rows': []}

### port_cities sheet

In [20]:
port_cities['port_code'].sort_values().unique()

array([' ogx', 'aov', 'axd', 'beh', 'bse', 'bwm', 'byv', 'bzj', 'cac',
       'cli', 'cma', 'cmo', 'cpi', 'ctm', 'cwg', 'dhj', 'dii', 'eak',
       'ebu', 'efq', 'ejl', 'elk', 'ffo', 'fhq', 'fmq', 'fwd', 'fzt',
       'gbo', 'ggv', 'ghd', 'gol', 'jgt', 'jit', 'jrw', 'jvz', 'kka',
       'kln', 'kzf', 'lbe', 'lbh', 'lcy', 'lou', 'mfh', 'miu', 'mix',
       'mnb', 'moi', 'mtu', 'ngb', 'nke', 'nmr', 'nsu', 'odk', 'ofl',
       'oyh', 'pkg', 'ply', 'pmh', 'ppp', 'prg', 'pwl', 'pwx', 'qam',
       'qhw', 'qlx', 'qpx', 'qso', 'qwh', 'qyr', 'rev', 'rfb', 'rnd',
       'sdg', 'suv', 'tby', 'tct', 'tpr', 'tsh', 'uzl', 'vco', 'vud',
       'wcp', 'wiy', 'wme', 'wqg', 'wrb', 'xhn', 'xjz', 'xya', 'ydd',
       'ymj', 'yqa', 'yrj', 'yub', 'ywp', 'zkn', 'zku', 'zpr', 'zwg',
       'zwo', 'zzk', 'zzz'], dtype=object)

In [21]:
# Removing whitespaces
port_cities['port_code'] = port_cities['port_code'].str.strip()

In [22]:
# Ports named after cities need to be checked manually
port_city_error = port_cities[port_cities['city'] == port_cities['port_name']].sort_values(by='city')
if len(port_city_error) > 0:
  typos["file_name"].append('Ports_ceties_codes_and_IDs')
  typos["column_name"].append('port name = city name ')
  typos["typos_rows"].append((port_city_error.index.values) + 2)

### countries_ids sheet

In [23]:
countries_ids['country'].sort_values().unique()

array(['american samoa', 'andorra la vella', 'anguilla',
       'antigua and barbuda', 'aruba', 'bahamas', 'barbados', 'belize',
       'bermuda', 'bhutan', 'british virgin islands', 'brunei',
       'cape verde', 'cayman islands', 'cook islands', 'curacao',
       'dominica', 'falkland islands', 'faroe islands', 'fiji',
       'french guiana', 'french polynesia', 'gibraltar', 'greenland',
       'grenada', 'guadeloupe', 'guam', 'guernsey', 'guyana', 'jersey',
       'kiribati', 'macau', 'maldives', 'malta', 'marshall islands',
       'martinique', 'mayotte', 'micronesia', 'monaco', 'montenegro',
       'montserrat', 'nauru', 'new caledonia', 'niue',
       'northern mariana islands', 'palau', 'qwer', 'reunion',
       'saint barthelemy', 'saint kitts and nevis', 'saint lucia',
       'saint martin', 'saint pierre and miquelon',
       'saint vincent and the grenadines', 'samoa', 'san marino',
       'sao tome and principe', 'seychelles', 'sint maarten',
       'solomon islands', 'suri

In [24]:
if len(typos['file_name']) > 0:
  typos_df = pd.DataFrame(typos, index=np.arange(len(typos['file_name'])))

### aug_sheet

In [25]:
# Creating a dictionary for possible aug_sheet typos
aug_sheet_typos = {'sheet_name': [], 'error_type': [], 'typos_seq': []}

#### trip_type

In [26]:
unknown_trip_types = aug_sheet[aug_sheet['trip_type'] == 'Unknown']
if len(unknown_trip_types) > 0:
  aug_sheet_typos["sheet_name"].append(unknown_trip_types['country'].values[0])
  aug_sheet_typos["error_type"].append('Unknown trip type')
  aug_sheet_typos["typos_seq"].append(unknown_trip_types.index.values+1)

#### country

In [27]:
# Checking countries errors
aug_new_countries = [country for country in aug_sheet['country'].unique()\
                    if country not in countries_ids['country'].unique()] # There are new countries
aug_new_countries

['paramaribo', 'isle of man', 'majuro']

In [28]:
# Replace capital names with the correct countries
aug_sheet['country'].replace({
  'paramaribo' : 'suriname',
  'majuro' : 'marshall islands'
}, inplace= True)

In [29]:
# Checking countries errors (again)
aug_new_countries = [country for country in aug_sheet['country'].unique()\
                    if country not in countries_ids['country'].unique()]
aug_new_countries

['isle of man']

In [30]:
for country in aug_new_countries:
  aug_sheet_typos["sheet_name"].append(country)
  aug_sheet_typos["error_type"].append('''country name wasn't in countries sheet''')
  aug_sheet_typos["typos_seq"].append('None')

In [31]:
# Adding new countries to countries_ids sheet
if len(aug_new_countries) > 0:
  aug_new_countries_df = {'country': [], 'country_id': []}
  for i, country in enumerate(aug_new_countries):
    aug_new_countries_df['country'].append(country)
    aug_new_countries_df['country_id'].append(countries_ids['country_id'].max() + i + 1)

  aug_new_countries_df = pd.DataFrame(
    aug_new_countries_df,
    index= np.arange(len(aug_new_countries))
  )

  countries_ids = pd.concat([countries_ids, aug_new_countries_df], ignore_index= True)

In [32]:
countries_ids['country'].sort_values().unique()

array(['american samoa', 'andorra la vella', 'anguilla',
       'antigua and barbuda', 'aruba', 'bahamas', 'barbados', 'belize',
       'bermuda', 'bhutan', 'british virgin islands', 'brunei',
       'cape verde', 'cayman islands', 'cook islands', 'curacao',
       'dominica', 'falkland islands', 'faroe islands', 'fiji',
       'french guiana', 'french polynesia', 'gibraltar', 'greenland',
       'grenada', 'guadeloupe', 'guam', 'guernsey', 'guyana',
       'isle of man', 'jersey', 'kiribati', 'macau', 'maldives', 'malta',
       'marshall islands', 'martinique', 'mayotte', 'micronesia',
       'monaco', 'montenegro', 'montserrat', 'nauru', 'new caledonia',
       'niue', 'northern mariana islands', 'palau', 'qwer', 'reunion',
       'saint barthelemy', 'saint kitts and nevis', 'saint lucia',
       'saint martin', 'saint pierre and miquelon',
       'saint vincent and the grenadines', 'samoa', 'san marino',
       'sao tome and principe', 'seychelles', 'sint maarten',
       'solomon 

#### Port errors

In [33]:
aug_sheet_ports = np.unique(aug_sheet['port_from'].tolist() + aug_sheet['port_to'].tolist())
aug_sheet_ports

array(['aalborg', 'alicante', 'antwerpen', 'antwrpen', 'avilés',
       'bensersiel', 'berwyn', 'bilbao', 'brake', 'bremen',
       'cala sabina (formentera)', 'carboneras', 'cartagena', 'castellón',
       'civitavecchia', 'cádiz', 'emdan', 'emden', 'fos-sur-mer',
       'frederikshavn', 'frederikshevn', 'frederykshavn', 'friderikshavn',
       'frm langeoog', 'helsingør', "helsingør'", 'hirtshals', 'kavala',
       'københavn', 'langeoog', "liepa'ja", 'liepaja', 'lübeck',
       'miami beach', 'milazzo', 'motril', 'norddeich', 'nordenham',
       'norderney', 'oostende', 'palma de mallorca', 'perth amboy',
       'puttgarden', 'sagunto', 'sjaellands odde ferry port',
       'thessaloniki', 'union city', 'wilhelmshaven', 'wismar',
       'zeebrugg', 'zeebrugge', 'zzzzzzzzzzzzz'], dtype='<U26')

In [34]:
aug_new_ports = [port for port in aug_sheet_ports\
                if port not in port_cities['port_name'].unique()]
aug_new_ports

['antwrpen',
 'berwyn',
 'emdan',
 'frederikshevn',
 'frederykshavn',
 'friderikshavn',
 'frm langeoog',
 "helsingør'",
 "liepa'ja",
 'liepaja',
 'miami beach',
 'perth amboy',
 'sjaellands odde ferry port',
 'zeebrugg',
 'zzzzzzzzzzzzz']

In [35]:
replacing_dict = {
  'antwrpen': 'antwerpen',
  'berwyn': 'heltermaa',
  'emdan' : 'emden',
  'frederikshevn': 'frederikshavn',
  'frederykshavn': 'frederikshavn',
  'friderikshavn': 'frederikshavn',
  'frm langeoog': 'langeoog',
  "helsingør'": 'helsingør',
  "liepa'ja": 'liepāja',
  'liepaja': 'liepāja',
  'miami beach': 'zeebrugge',
  'perth amboy': 'oostende',
  'sjaellands odde ferry port': 'sjællands odde ferry port',
  'union city': 'aalborg',
  'zeebrugg': 'zeebrugge'
}

for col in ['port_from', 'port_to']:
  aug_sheet[col].replace(replacing_dict, inplace=True)

In [36]:
aug_sheet_ports_2 = np.unique(aug_sheet['port_from'].tolist() + aug_sheet['port_to'].tolist())

aug_new_ports_2 = [port for port in aug_sheet_ports_2\
                if port not in port_cities['port_name'].unique()]
aug_new_ports_2

['zzzzzzzzzzzzz']

In [37]:
# Before cleaning
len(aug_sheet_ports)

52

In [38]:
# After cleaning
len(aug_sheet_ports_2)

40

In [39]:
if len(aug_new_ports_2) > 0:
  aug_new_ports_df = {'city': [], 'port_name': [], 'port_code': [], 'port_id': []}

  # return city for the target port
  aug_new_ports_city = [aug_sheet[(aug_sheet['port_from'] == port) | (aug_sheet['port_to'] == port)]\
                          ['country'].values[0] for port in aug_new_ports_2]

  # Adding new ports to the main sheet
  for i, port in enumerate(aug_new_ports_2):
    aug_new_ports_df['city'].append(aug_new_ports_city[i])
    aug_new_ports_df['port_name'].append(aug_new_ports_2[i])
    aug_new_ports_df['port_code'].append(aug_new_ports_2[i][0:3])
    aug_new_ports_df['port_id'].append(port_cities['port_id'].max() + i + 1)

  # Adding new ports to errors sheet
  for i, port in enumerate(aug_new_ports_2):
    target_row = aug_sheet[(aug_sheet['port_from'] == port) | (aug_sheet['port_to'] == port)]
    aug_sheet_typos["sheet_name"].append(target_row['country'].values[0])
    aug_sheet_typos["error_type"].append('''port name wasn't in ports sheet''')
    aug_sheet_typos["typos_seq"].append(target_row.index.values+1)

  port_cities = pd.concat([port_cities, pd.DataFrame(aug_new_ports_df)], ignore_index= True)

In [40]:
if len(aug_sheet_typos['sheet_name']) > 0:
  aug_sheet_typos_df = pd.DataFrame(aug_sheet_typos, index=np.arange(len(aug_sheet_typos['sheet_name'])))

# Check cleaning results

In [41]:
print(aug_sheet.head(2))
print(" ")
print(countries_ids.head(2))
print(" ")
print(port_cities.head(2))

   country      port_from      port_to trip_type  price
0  reunion  frederikshavn      liepāja         O   3306
1  reunion          emden  fos-sur-mer         O   6363
 
    country  country_id
0   reunion         100
1  suriname          40
 
         city port_name port_code  port_id
0  guttenberg  a coruña       ogx      870
1  union city   aalborg       ffo      229


# Preparing data in the required form

In [42]:
prep_df = aug_sheet

# First two columns
prep_df['port_from_id'] = aug_sheet['port_from'].apply(
  lambda x: port_cities[port_cities['port_name'] == x ]['port_id'].values[0]
)
prep_df['port_to_id'] = aug_sheet['port_to'].apply(
  lambda x: port_cities[port_cities['port_name'] == x ]['port_id'].values[0]
)

# port codes
prep_df['code_from'] = aug_sheet['port_from'].apply(
  lambda x: port_cities[port_cities['port_name'] == x ]['port_code'].values[0]
)
prep_df['code_to'] = aug_sheet['port_to'].apply(
  lambda x: port_cities[port_cities['port_name'] == x ]['port_code'].values[0]
)

# URL
prep_df['url'] = prep_df['code_from'] + '-'\
  + prep_df['code_to'] + '-' + prep_df['trip_type'].str.lower()

# dates
prep_df['publish_date'] = date.today()
prep_df['expire_date'] = date.today() + timedelta(days=35)

prep_df.head()

,country,port_from,port_to,trip_type,price,port_from_id,port_to_id,code_from,code_to,url,publish_date,expire_date
0,reunion,frederikshavn,liepāja,O,3306,372,32,cac,zzk,cac-zzk-o,2024-05-03,2024-06-07
1,reunion,emden,fos-sur-mer,O,6363,572,248,rev,kka,rev-kka-o,2024-05-03,2024-06-07
2,reunion,zeebrugge,bensersiel,O,3666,903,696,ymj,tby,ymj-tby-o,2024-05-03,2024-06-07
3,reunion,emden,sagunto,O,6166,572,811,rev,bwm,rev-bwm-o,2024-05-03,2024-06-07
4,reunion,emden,milazzo,O,3161,572,913,rev,jvz,rev-jvz-o,2024-05-03,2024-06-07


In [43]:
def convert_to_arabic(number):
  english_to_arabic = {
    0: "٠",
    1: "١",
    2: "٢",
    3: "٣",
    4: "٤",
    5: "٥",
    6: "٦",
    7: "٧",
    8: "٨",
    9: "٩"
  }
  arabic_number = ""
  for digit in str(number):
    arabic_number += english_to_arabic[int(digit)]
  return arabic_number

In [44]:
offer_content = prep_df[['url', 'country' , 'price']]\
  .rename(columns={'url': 'offer_url', 'country': 'country_id'})

offer_content['country_id'] = offer_content['country_id'].apply(
  lambda x: countries_ids[countries_ids['country'] == x]['country_id'].values[0]
)

offer_content['price'] = offer_content['price'].apply(
  lambda x: f'{{"en":"{x}","ar":"{convert_to_arabic(x)}","gr":"{x}","it":"{x}","cz":"{x}","fr":"{x}","sk":"{x}"}}'
)

offer_content.info()

<class 'pandas.core.frame.DataFrame'>
Index: 180 entries, 0 to 17
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   offer_url   180 non-null    object
 1   country_id  180 non-null    int64 
 2   price       180 non-null    object
dtypes: int64(1), object(2)
memory usage: 5.6+ KB


In [45]:
offer_content.to_excel('offer_content.xlsx', index=False)

In [46]:
offers_template = prep_df[['port_from_id', 'port_to_id', 'trip_type', 'url']].drop_duplicates().reset_index(drop= True)
offers_template['publish_date'] = date.today()
offers_template['expire_date'] = date.today() + timedelta(days= 35)
offers_template.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   port_from_id  54 non-null     int64 
 1   port_to_id    54 non-null     int64 
 2   trip_type     54 non-null     object
 3   url           54 non-null     object
 4   publish_date  54 non-null     object
 5   expire_date   54 non-null     object
dtypes: int64(2), object(4)
memory usage: 2.7+ KB


In [47]:
offers_template.to_excel('offers_template.xlsx', index=False)

In [48]:
# Creating extra file to report new countries, ports errors
writer = pd.ExcelWriter('possible_errors.xlsx', engine='xlsxwriter')

if len(duplicates_df) > 0:
  duplicates_df.to_excel(writer, sheet_name='duplicates',index= False)

if len(port_city_error) > 0:
  typos_df.to_excel(writer, sheet_name='backend_typos',index= False)

if len(aug_sheet_typos['sheet_name']) > 0:
  aug_sheet_typos_df.to_excel(writer, sheet_name='target_file_typos',index= False)

writer.close()